<center>
<img style="float: right;" src="https://www.isprs.org/congresses/nice2020-2022/isprsNice22-logo.jpg" alt="drawing" width="150"/>

<img style="float: right;" src="https://www.isprs2022-nice.com/wp-content/uploads/2019/03/isprs-logoXS_150_150_Circle3.png" alt="drawing" width="150"/>

<img style="float: right;" src="https://github.com/CNES/cars/raw/master/docs/source/images/picto_transparent_mini.png" alt="drawing" width="150"/>

</center>

<center> <h1> CARS, a satellite multi view stereo pipeline (half day – morning)</h1> </center>

<h6> From stereo images </h6> |  <h6> CARS produces a Digital Surface Model (DSM) </h6>
:-------------------------:|:-------------------------:
<img style="float: right;" src="https://github.com/CNES/cars/raw/master/docs/source/images/animation_sat.gif" alt="drawing" height="200"/> |  <img style="float: right;" src="https://github.com/CNES/cars/raw/master/docs/source/images/overview_dsm_3d.gif" alt="drawing" height="200"/>

### Quick installation (<10min)

In [ ]:
!wget https://raw.githubusercontent.com/dyoussef/otb-colab/main/cars-in-colab.sh
!bash cars-in-colab.sh

In [2]:
import os
# otb environment variables
os.environ["OTB_APPLICATION_PATH"] = "/usr/lib/otb/applications"
os.environ["OTB_MAX_RAM_HINT"] = "2000"
os.environ["OTB_LOGGER_LEVEL"] = "WARNING"

### Download open data pyramids

In [ ]:
!wget https://raw.githubusercontent.com/CNES/cars/master/docs/source/demo/data_samples.tar.bz2
!tar xvfj data_samples.tar.bz2

### Two steps (prepare + compute_dsm), one DSM

In [ ]:
!cars prepare -i data_samples/input12.json -o out/prepare12

In [ ]:
!cars prepare -i data_samples/input13.json -o out/prepare13

In [ ]:
!cars compute_dsm -i out/prepare12/content.json out/prepare13/content.json -o out/compute

### Let's see the results

#### Open & format output data

In [7]:
import rasterio as rio
import numpy as np

with rio.open('out/compute/dsm.tif') as dsm_reader:
  altitudes = dsm_reader.read(1)
  transform = dsm_reader.transform
  width, height = dsm_reader.width, dsm_reader.height
  cols, rows = np.meshgrid(np.arange(width), np.arange(height))
  
  # get coordinates to plot points cloud
  x_coords, y_coords = rio.transform.xy(transform, rows, cols, offset='center')
  x_coords = np.ravel(x_coords).T
  y_coords = np.ravel(y_coords).T
  z_coords = altitudes.reshape(-1).T
  
  nodata = dsm_reader.nodata

with rio.open('out/compute/clr.tif') as clr_reader:
  colors = clr_reader.read().astype(float)
  # swap axes for imshow
  colors = np.swapaxes(colors[:3, :, :], 0, 2)

  # rescale colors
  for idx in range(3):
    colors[..., idx] = colors[..., idx].astype(float) / colors[..., idx].max()

# stack coords as points cloud
cloud = np.stack((x_coords, y_coords, z_coords), axis=1)
valid = cloud[:, 2] != nodata
cloud = cloud[valid]

# remove nodata altitudes
altitudes[altitudes==nodata] = np.nan

#### Imshow (2D)

In [ ]:
# plot altitudes + colors
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 9), constrained_layout=True)
im1 = ax1.imshow(altitudes.T, cmap='jet', aspect="auto")
ax1.axis('off')
ax1.set_title('altitudes', fontsize=15)
ax2.imshow(colors, aspect="auto")
ax2.axis('off')
ax2.set_title('colors', fontsize=15)
fig.colorbar(im1, ax=ax1, shrink=0.7, aspect=30, orientation="horizontal", label="meters")
plt.show()

#### Scatter (3D)

In [ ]:
# plot 3d points cloud (subsampled)
import plotly.graph_objects as go

inds = np.random.choice(range(cloud.shape[0]), 20000)
x = cloud[inds, 0]
y = cloud[inds, 1]
z = cloud[inds, 2]

layout = go.Layout(scene=dict(aspectmode="data"))
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(
                                   size=2,
                                   color=z,
                                   colorscale='jet'))],
                layout=layout)
fig.show()